# 10. Gépi tanulás – gyakorlat

>Számítógépes nyelvészet, 2018 tavasz

>Simon Eszter, Mittelholcz Iván

>MTA, Nyelvtudományi Intézet

## 1. Scikit-learn

TODO

## 2. Huntag3

### 2.1. Előkészületek

* [HunTag3](https://github.com/ppke-nlpg/HunTag3) letöltése
* kitömörítése

In [ ]:
%%bash

wget -q -c https://github.com/ppke-nlpg/HunTag3/archive/master.zip
unzip -q -u master.zip

### 2.2. Adatok

Forrás: http://hlt.sztaki.hu/resources/hunnerwiki.html

Oszlopok:

1. **token**
2. egyértelműsített morfológiai elemzés (KR kód)
3. lemma
4. **gold standard**

In [ ]:
!head -n 13 huwiki.ner.sample.train.tsv

### 2.3. Features

A *feature*-öket *on the fly* generáljuk a tanítás során rövid függvényekkel, melyek bemenetüket a TSV valamelyik oszlopából vehetik.

Ezen függvények helye a *features.py* fájlban van. A HunTag3-mal jön egy alap *features.py*, ezt módosíthatjuk, vagy sajátot is írhatunk.

Hogy milyen featur-öket akarunk használni, és melyik feature a TSV melyik oszlopát használja, azt az általunk írt konfigfájlban tudjuk megadni.

#### konfig

A konfigurációs beállításokat [YAML](https://en.wikipedia.org/wiki/YAML) formátumban kell megadni.

* *name*: a *most-informative-features* kimenetében ezen a néven jelenik meg a *feature*
* *type*: lehet *token*, *sentence*, vagy *lex*, ezek a feature-ök függvényeinek is prefixei egyben
* *actionName*: a függvény neve, *type* prefix nélkül
    * *lex*: lexikai jegynél a lexikon elérési útját kell megadni
* *fields*: TSV oszlop száma (0-ával kezdődik)

In [ ]:
%%writefile HunTag3-master/myconf.yaml

%YAML 1.1
---
default:
  cutoff: 1  #  1 if not set
  radius: 3  # -1 if not set

features:
 - # szóalak
    name: token
    type: token
    actionName: getToken
    fields: 0
 - # morf elemzés
    name: ana
    type: token
    actionName: getAna
    fields: 1
 - # csupa nagybetű-e
    name: isUpper
    type: token
    actionName: isUpper
    fields: 0
# - # org lista
#    name: orglist
#    type: lex
#    actionName: hunner/lex_from_Eszter.Simon/org.fromtraincorpus.lex
#    fields: 0
# - # loc lista
#    name: loclist
#    type: lex
#    actionName: hunner/lex_from_Eszter.Simon/loc.fromtraincorpus.lex
#    fields: 0
# - # misc lista
#    name: misclist
#    type: lex
#    actionName: hunner/lex_from_Eszter.Simon/misc.fromtraincorpus.lex
#    fields: 0
# - # per lista
#    name: perlist
#    type: lex
#    actionName: hunner/lex_from_Eszter.Simon/per.fromtraincorpus.lex
#    fields: 0
...



#### features

* egy feature-függvénynek listával kell visszatérnie
* a *bool* típusú visszatérési értékeket konvertálni kell *int*-re

In [ ]:
%%writefile HunTag3-master/features.py

def token_isUpper(token, _=None):
    return [int(token.isupper())]

def token_getToken(token, _=None):
    return [token]

def token_getAna(ana, _=None):
    return [ana]


### 2.4. Tanítás

#### megfigyelési modell

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py train -i ../huwiki.ner.sample.train.tsv -c myconf.yaml -m NER
ls -l NER.model

#### átmenet modell

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py transmodel-train -i ../huwiki.ner.sample.train.tsv -m NER

### 2.5. Címkézés, tesztelés, kiértékelés

### címkézés

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py tag -i ../huwiki.ner.sample.test.tsv -c myconf.yaml -m NER -o huwiki.ner.test.tagged.tsv

#### kiértékelés

In [ ]:
%%bash

cd HunTag3-master/
python3 utils/eval.py -c -i huwiki.ner.test.tagged.tsv

### 2.6. Feladatok

Implementáljatok pár új feature-t és értékeljétek ki ezekkel a ner-taggert.
Az új feature-ök legyenek:

* szófaj (része az elemzésnek a 2. oszlopban)
* lemma (3. oszlop)
* nagybetűvel kezdődik-e
* van-e benne nagybetű

#### konfig

In [ ]:
%%writefile HunTag3-master/myconf.yaml

%YAML 1.1
---
default:
  cutoff: 1  #  1 if not set
  radius: 3  # -1 if not set

features:
 - # szóalak
    name: token
    type: token
    actionName: getToken
    fields: 0
 - # morf elemzés
    name: ana
    type: token
    actionName: getAna
    fields: 1
# - # szófajkód
#    name: pos
#    type: ...
#    actionName: getPos
#    fields: ...
#
# - # lemma
#    name: lem
#    type: ...
#    actionName: getLem
#    fields: ...
 - # csupa nagybetű-e
    name: isUpper
    type: token
    actionName: isUpper
    fields: 0
# - # nagybetűvel kezdődik-e
#    name: isTitle
#    type: ...
#    actionName: isTitle
#    fields: ...
# - # van-e benne nagybetű
#    name: hasUpper
#    type: ...
#    actionName: hasUpper
#    fields: ...
...



#### features

In [ ]:
%%writefile HunTag3-master/features.py


def token_getToken(token, _=None):
    return [token]

def token_getAna(ana, _=None):
    return [ana]

#def token_getPos(ana, _=None):
#    return ...

#def token_getLem(lem, _=None):
#    return ...

def token_isUpper(token, _=None):
    return [int(token.isupper())]

#def token_isTitle(token, _=None):
#    return ...

#def token_hasUpper(token, _=None):
#    return ...



#### megfigyelési modell

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py train -i ../huwiki.ner.sample.train.tsv -c myconf.yaml -m NER
ls -l NER.model

#### átmenet modell

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py transmodel-train -i ../huwiki.ner.sample.train.tsv -m NER

#### címkézés

In [ ]:
%%bash

cd HunTag3-master/
python3 huntag.py tag -i ../huwiki.ner.sample.test.tsv -c myconf.yaml -m NER -o huwiki.ner.test.tagged.tsv

#### kiértékelés

In [ ]:
%%bash

cd HunTag3-master/
python3 utils/eval.py -c -i huwiki.ner.test.tagged.tsv